## Import Packages

In [1]:
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../utils")

from timeslots import timeslot_times
import utils_functions
from unavailability import group1_chairs, group1_members, group1_est_unavailability, group2_chairs, group2_members, group2_est_unavailability

## Import Optimal Data

In [2]:
group1_optimal = utils_functions.read_optimal_txt("../data/group1_results.txt")
group2_optimal = utils_functions.read_optimal_txt("../data/group2_results.txt")

In [3]:
# Table Colours.
header_grey = "#ADADAD"
break_grey = "#D3D3D3"

## Schedule Generator Functions

In [23]:
# Creates table mapping for a person.
def person_availability_mapping(optimal_sol, person_index, group_unavailability):
    person_availability = {}
    person_busy = group_unavailability[person_index]

    for pattern in optimal_sol:
        if pattern[1][person_index] == 1:
            person_availability[pattern[0]] = ("Meeting", "red")

    personal_busy = utils_functions.calculate_unavailability([[person_busy[0]]])
    for slot in personal_busy[0]:
        person_availability[slot] = ("Busy", "yellow")

    break_busy = utils_functions.calculate_unavailability([[person_busy[1], person_busy[2]]])
    for slot in break_busy[0]:
        person_availability[slot] = ("BREAK", break_grey)

    lunch_busy = utils_functions.calculate_unavailability([[person_busy[3]]])
    for slot in lunch_busy[0]:
        person_availability[slot] = ("LUNCH", break_grey)

    for time in range(1, 171):
        if time not in person_availability:
            person_availability[time] = (None, "white")
    return person_availability

In [25]:
# Setup table data for a person.
def setup_table_schedule(person_title, availability):
    table_data = [
        [(person_title, "white"), ("Monday", header_grey), ("Tuesday", header_grey), ("Wednesday", header_grey), ("Thursday", header_grey), ("Friday", header_grey)],
    ]
    for i, time in enumerate(timeslot_times):
        row_data = [(time, header_grey)]
        for day in range(0, 5):
            cell = (i + 1) + (day * len(timeslot_times))
            row_data.append(availability[cell])
        table_data.append(row_data)
    return table_data

In [26]:
# Save schedules as images.
def save_group_schedules(group_people, optimal_sol, group_unavailability):
    for i, person in enumerate(group_people):
        person_title = f"Chair {person}" if i < 5 else f"Member {person}"
        person_availability = person_availability_mapping(optimal_sol, i, group_unavailability)
        table_data = setup_table_schedule(person_title, person_availability)
        text_data = [[cell[0] for cell in row] for row in table_data]
        colour_data = [[cell[1] for cell in row] for row in table_data]

        # Save schedule.
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.axis("off")
        table = ax.table(cellText=text_data, cellColours=colour_data, loc="center", cellLoc="center")
        plt.savefig(f"../data/schedules/{person_title}.png", bbox_inches="tight")
        plt.close()


## Save Schedules

In [27]:
group1_people = [*group1_chairs, *group1_members]
group2_people = [*group2_chairs, *group2_members]

save_group_schedules(group1_people, group1_optimal, group1_est_unavailability)
save_group_schedules(group2_people, group2_optimal, group2_est_unavailability)
